In [1]:
import pandas as pd
import numpy as np
%matplotlib inline 
import spacy
import re
from functools import reduce
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn import ensemble
from sklearn.model_selection import train_test_split

from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer

pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)


height has been deprecated.



In [2]:
# Data obtained from Figure Eight.  Contains economic news articles that were categorized as either having information on the US economy or not.
data = pd.read_csv("Economic-News.csv",encoding = "ISO-8859-1")
#link: https://www.figure-eight.com/wp-content/uploads/2016/03/Full-Economic-News-DFE-839861.csv

In [3]:
# See a few rows of the data
data.head()

,_unit_id,_golden,_unit_state,_trusted_judgments,_last_judgment_at,positivity,positivity:confidence,relevance,relevance:confidence,articleid,date,headline,positivity_gold,relevance_gold,text
0,842613455,False,finalized,3,12/5/15 17:48,3.0,0.6400,yes,0.640,wsj_398217788,8/14/91,Yields on CDs Fell in the Latest Week,NaN,NaN,NEW YORK -- Yields on most certificates of dep...
1,842613456,False,finalized,3,12/5/15 16:54,NaN,NaN,no,1.000,wsj_399019502,8/21/07,The Morning Brief: White House Seeks to Limit ...,NaN,NaN,The Wall Street Journal Online</br></br>The Mo...
2,842613457,False,finalized,3,12/5/15 1:59,NaN,NaN,no,1.000,wsj_398284048,11/14/91,Banking Bill Negotiators Set Compromise --- Pl...,NaN,NaN,WASHINGTON -- In an effort to achieve banking ...
3,842613458,False,finalized,3,12/5/15 2:19,NaN,0.0000,no,0.675,wsj_397959018,6/16/86,Manager's Journal: Sniffing Out Drug Abusers I...,NaN,NaN,The statistics on the enormous costs of employ...
4,842613459,False,finalized,3,12/5/15 17:48,3.0,0.3257,yes,0.640,wsj_398838054,10/4/02,Currency Trading: Dollar Remains in Tight Rang...,NaN,NaN,NEW YORK -- Indecision marked the dollar's ton...


In [4]:
# See data shape
data.shape

(8000, 15)

In [5]:
# Check for imbalance
data["relevance"].value_counts()


no          6571
yes         1420
not sure       9
Name: relevance, dtype: int64

In [6]:
#Separate data into relevant and not relevant for parsing
relevant = data[data["relevance"] == "yes"]
not_relevant = data[data["relevance"] == "no"]
relevant = relevant.iloc[0:500,]
not_relevant = not_relevant.iloc[0:500,]
yes_text = relevant["text"]
no_text = not_relevant["text"]

In [7]:
#Create list to see samples of data to determine what cleaning is needed
text= data["text"]
relevance = data["relevance"]
text_list= (1,100,500,1000,1500,2250,2500,3000,3350,3400,4500)

In [8]:
# See text to determine cleaning needed.
for headline in text_list:
    print (text[headline][:50]) #Remove slice to see full data while determining what cleaning is needed

The Wall Street Journal Online</br></br>The Mornin
J.P. Morgan Chase & Co. reached a tentative deal t
Five large banks reported improved second-quarter 
SANTA CLARA, Calif. -- George Bush played down the
It's no secret that the tea party faithful regard 
AMSTERDAM--Dutch retailer Royal Ahold NV said Wedn
Behind every good rally these days is a good deal 
South Korea's efforts to engineer an economic reco
PLATTSBURGH, N.Y. -- The rise of conservative "tea
{First of a Series}</br></br>W.J. Turner L. Cobden
Regan, in his strongest language to date, blamed b


In [9]:
# Clean the text
def text_cleaner(text):
    text = text.apply(lambda x:re.sub("</br></br>", "", x))
    text = text.apply(lambda x:re.sub("---", "", x))
    text = text.apply(lambda x:re.sub("--", "", x))
    text = text.apply(lambda x:re.sub("<", "", x))
    text = text.apply(lambda x:re.sub(";", "", x))
    text = text.apply(lambda x:re.sub(";", "", x))
    text = text.apply(lambda x:re.sub("ÛÏ", "", x))
    text = text.apply(lambda x:re.sub("Ûª", "", x))
    text = text.apply(lambda x:re.sub("Û", "", x))
    text = text.apply(lambda x:re.sub("ûó", "", x))
    text = text.apply(lambda x:re.sub("ÛÓ", "", x))
    text = text.apply(lambda x:re.sub("\x89ûó", "", x))
    text = text.apply(lambda x:re.sub("x89ûó", "", x))
    text = text.apply(lambda x:re.sub("\x89ðê", "", x))  
    text = text.apply(lambda x:re.sub("Author:", "", x))
    text = text.apply(lambda x:re.sub("By:", "", x))
    return text
cleaned_text=text_cleaner(text)
yes_cleaned = text_cleaner(yes_text)
no_cleaned = text_cleaner(no_text)
    

In [32]:
# See cleaned text and check if additional cleaning needed
for headline in text_list:
    print (cleaned_text[headline][:50]) #Remove slice if needed to see full data

The Wall Street Journal OnlineThe Morning Brief, a
J.P. Morgan Chase & Co. reached a tentative deal t
Five large banks reported improved second-quarter 
SANTA CLARA, Calif.  George Bush played down the n
It's no secret that the tea party faithful regard 
AMSTERDAMDutch retailer Royal Ahold NV said Wednes
Behind every good rally these days is a good deal 
South Korea's efforts to engineer an economic reco
PLATTSBURGH, N.Y.  The rise of conservative "tea p
{First of a Series}W.J. Turner L. Cobden, a top mu
Regan, in his strongest language to date, blamed b


In [11]:
#Combine rows into one string for Spacy parsing.
yes_text_combined = reduce(lambda x, y: x+y, yes_cleaned)
yes_text_combined = yes_text_combined.lower()


no_text_combined = reduce(lambda x, y: x+y, no_cleaned)
no_text_combined = no_text_combined.lower()


In [12]:
#Load English Spacy
nlp = spacy.load("en")

In [13]:
#Increase max length for nlp.  Then run Spacy parsing on relevant and not relevant
nlp.max_length = 11000000
yes_nlp = nlp(yes_text_combined)
no_nlp = nlp(no_text_combined)

In [14]:
#Convert parsed text to sentences
yes_sents = [[sent,"Relevant"] for sent in yes_nlp.sents]
no_sents = [[sent,"Not_Relevant"] for sent in no_nlp.sents]
sentences = pd.DataFrame(yes_sents + no_sents)
nlp_text =sentences
sentences.head(10)

,0,1
0,"(new, york, , yields, on, most, certificates,...",Relevant
1,"(two, banks, in, the, banxquote, survey, ,, ci...",Relevant
2,"(against, the, yen, ,, the, dollar, was, tradi...",Relevant
3,"(against, the, swiss, franc, ,, the, dollar, w...",Relevant
4,"(17.weighing, on, the, downside, were, alcoa, ...",Relevant
5,"(hewlett, -, packard, dropped, 21, cents, ,, o...",Relevant
6,"(the, decline, continued, in, new, york, tradi...",Relevant
7,"(lower, bond, prices, translate, into, higher,...",Relevant
8,"(the, bond, market, later, closed, little, -, ...",Relevant
9,"(,, new, york.in, late, new, york, trading, ,,...",Relevant


In [15]:
# Word counter function
def word_counter(text, include_stop=True):
        words=[]
        for token in text:
            if not token.is_punct and (not token.is_stop or include_stop):
                words.append(token.text)
        return Counter(words)

In [16]:
# Word frequency for relevant text
word_frequency_relevant = word_counter(yes_nlp, include_stop=False).most_common(20)
print(word_frequency_relevant)

[("'s", 1008), ('$', 593), ('said', 517), ('market', 475), ('u.s', 471), ('year', 430), (' ', 406), ('prices', 375), ('economy', 369), ('rate', 355), ('rates', 345), ('economic', 337), ('interest', 326), ('fed', 305), ('new', 303), ('stock', 278), ('federal', 271), ('dollar', 265), ('quarter', 259), ('growth', 250)]


In [17]:
# Word frequency for non-relevant
word_frequency_not_relevant = word_counter(no_nlp, include_stop=False).most_common(20)
print(word_frequency_not_relevant)

[("'s", 1157), ('$', 635), ('u.s', 454), (' ', 446), ('market', 441), ('said', 441), ('year', 369), ("n't", 357), ('stock', 339), ('new', 307), ('rates', 260), ('investors', 240), ('billion', 233), ('interest', 233), ('economic', 229), ('economy', 221), ('federal', 217), ('rate', 215), ('million', 214), ('mr', 212)]


In [18]:
# Function to count lemmas
def lemma_frequencies(text, include_stop=True):
    lemmas = []
    for token in text:
        if not token.is_punct and (not token.is_stop or include_stop):
            lemmas.append(token.lemma_)
    return Counter(lemmas)

In [19]:
#See most common lemmas for relevant text
text_lemma_freq_relevant = lemma_frequencies(yes_nlp, include_stop=False).most_common(10)
print(text_lemma_freq_relevant)

[("'s", 875), ('rate', 700), ('say', 607), ('market', 596), ('$', 593), ('year', 550), ('price', 510), ('stock', 460), ('rise', 438), ('u.s', 414)]


In [20]:
#See most common lemmas
text_lemma_freq_not_relevant = lemma_frequencies(no_nlp, include_stop=False).most_common(10)
print(text_lemma_freq_not_relevant)

[("'s", 1016), ('$', 635), ('say', 601), ('market', 579), ('year', 505), ('stock', 483), ('rate', 475), (' ', 446), ('u.s', 379), ('not', 357)]


In [21]:
#Bag of words features
def bag_of_words(text):
    allwords = [token.lemma_ for token in text if not token.is_punct and not token.is_stop]
    return [item[0] for item in Counter(allwords).most_common(2000)]

def bow_features(sentences, common_words):
    bow_df = pd.DataFrame(columns=common_words)
    bow_df["text"] = sentences[0]
    bow_df["relevance"] = sentences[1]
    bow_df.loc[:,common_words] = 0
    for i, sentence in enumerate(bow_df["text"]):
        words = [token.lemma_ for token in sentence if (not token.is_punct and not token.is_stop and token.lemma_ in common_words)]
        for word in words:
            bow_df.loc[i, word] +=1
        if i % 500 == 0:
            print("Processing row {}".format(i))
    return bow_df

In [22]:
# Run bag of words on relevant and not relevant sets.  Then combine.
yes_words = bag_of_words(yes_nlp)
no_words = bag_of_words(no_nlp)
common_words = set(yes_words + no_words)

In [23]:
# Add words to dataframe then count for each row of text
word_counts = bow_features(sentences, common_words)
word_counts.head()

Processing row 0
Processing row 500
Processing row 1000
Processing row 1500
Processing row 2000
Processing row 2500
Processing row 3000
Processing row 3500
Processing row 4000
Processing row 4500
Processing row 5000
Processing row 5500
Processing row 6000
Processing row 6500


,somewhat,part,campaign,disappear,productive,index,launch,health,goal,turn,25,modest,initially,church,1986,workforce,witness,seven,selling,compound,toll,wholesale,alike,outstanding,3.4,daniel,bed,a.,presidential,exposure,status,hire,war,introduce,fail,broker,reduction,insist,recovery.the,2.9,inquiry,necessarily,safe,unusual,familiar,5.4,recovery,reject,meeting,treasury,1970,pause,uneven,intensify,airline,unemployment,link,thin,solvency,week.the,line,earlier.the,pound,seasonal,liberal,overnight,output,ignore,previous,tokyo,passage,slower,wal,publishing,administration,turmoil,banxquote,sector,expenditure,joseph,10-year,example,operating,deadline,making,apparel,lag,increasingly,friday,especially,home,hearing,agreement,calm,crist,arm,watcher,shake,ge,specific,curve,edward,ratio,commission,bellwether,indian,sour,cause,estate,share,comment,article,poor,take,slowdown,digit,lift,fannie,mystery,kudlow,woe,hong,free,calif,manufacturing,1.2,j.p,board,"200,000",manufacturer,relative,bad,despite,change,december,couple,anytime,anxiety,hop,jobless,science,threaten,janet,drift,need,india,retailer,certainly,testing,spot,transform,debate,oil,analyze,worry,video,shape,decent,weekly,michael,recently,38,survey,negative,journal,gate,seasonally,contribution,fiscal,retirement,northeast,28,obvious,ward,deal,canada,thomson,speculation,contract,fast,definitely,22,wall,platinum,implement,citigroup,2.5,1995,corner,chapter,holding,underscore,opposition,downbeat,core,insurer,service,reversal,marketing,half,pose,smooth,mark,human,extraordinary,ounce,reporter,snap,rush,historical,w.,proposal,approach,nature,rebound,escape,allege,upper,coalition,reflect,quantitative,gold,kick,37,"20,000",step,carrier,boston,bush,send,reinforce,coming,indicator,government,significantly,lender,competition,hurricane,knock,tiny,score,go,new,tighten,development,sterling,sizable,participation,round,forward,...,facility,principal,greenback,critic,6.3,lehman,revive,place,non,situation,fierce,benchmark,aug,sharply,money,speak,session,caution,drive,ahead,bell,victory,common,39,hang,trust,people,batter,3.1,chrysler,visit,germany,soon,oct,statistical,investigation,2,world,deposit,cloud,2010,m1,enormous,pia,far,course,separate,nominal,president,forecaster,mae,agenda,treat,fall,decision,1994,tight,picture,produce,narrow,east,liquidity,source,1.4,tool,im,subprime,minute,sponsor,relate,slump,creek,mix,legislative,category,brady,soft,train,mayor,october,decade,average,industry,optimistic,favor,benefit,guide,reluctance,bailout,weakness,institute,undermine,debut,flat,correct,be,4.8,lie,virtually,35,cheap,bureau,2001,squeeze,4,bull,tender,impose,bolster,count,ticket,age,access,small,24,internal,revenue,baker,publisher,asia,advancer,woman,demand,premium,own,steady,franc,1999,understand,consecutive,close,indicate,thailand,insurance,corp,schedule,greenspan,wave,mail,e,highlight,55,necessary,roger,feeling,track,politician,seriously,international,gear,basic,energy,crude,ceiling,restructuring,truck,sentiment,ian,variety,49,practice,month,emerge,shopper,client,publish,damage,3.2,southeast,chain,housing,pretty,clean,inc,invest,unchanged,system,anticipate,holiday,shy,anecdotal,investment,reach,conviction,earn,year,par,1981,well,middle,back,2.7,unprecedented,lackluster,alhadeff,stream,publication,hewlett,1988,trial,pickup,yen,spur,welcome,affect,reluctant,amount,rep,association,competitor,provision,chief,strategist,aggressively,1973,louis,regard,eighth,urge,adelphia,momentum,a.m.,sustain,14,d.,fraud,5.3,ease,median,college,deliver,restrain,declare,january,enthusiasm,anemic,evan,aide,frequency,improvement,thanksgiving,easy,600,tenure,maybe,40,employment,gop,text,relevance
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0

In [24]:
#Random forest model function
def random_forest_model(x_train,y_train,x_test,y_test):
    rfc = ensemble.RandomForestClassifier()
    model = rfc.fit(x_train, y_train)
    print('Training set score:', model.score(x_train, y_train))
    print('\nTest set score:', model.score(x_test, y_test))
    cross = cross_val_score(rfc,x_test, y_test, cv=3)
    print(cross)
    

In [25]:
# Bag of words train-test-split

y = word_counts['relevance']
x = np.array(word_counts.drop(['text','relevance'], 1))

x_train_bow, x_test_bow, y_train_bow, y_test_bow = train_test_split(x, y, test_size=0.4, random_state=0)

In [26]:
# Run random forest on bag of words features
random_forest_model(x_train_bow, y_train_bow, x_test_bow, y_test_bow)

Training set score: 0.979114868225

Test set score: 0.609246830723
[ 0.61521253  0.60514541  0.61409396]


In [27]:
#Create vectorizer with train-test-split

vectorizer = TfidfVectorizer(max_df=0.5, min_df=2, stop_words='english', lowercase=True, use_idf=False, norm=u'l1', smooth_idf=True)

vector_x_train, vector_x_test, vector_y_train, vector_y_test = train_test_split(cleaned_text, relevance, test_size=0.4, random_state=0)

In [28]:
#Vectorizer fit and transform.
text_tfidf = vectorizer.fit_transform(cleaned_text)

x_train_tfidf, x_test_tfidf = train_test_split(text_tfidf, test_size=.4, random_state=42)

x_train_tfidf_csr = x_train_tfidf.tocsr()

n = x_train_tfidf_csr.shape[0]

tfidf_bypara = [{} for _ in range(0,n)]

terms = vectorizer.get_feature_names()

for i, j in zip(*x_train_tfidf_csr.nonzero()):
    tfidf_bypara[i][terms[j]] = x_train_tfidf_csr[i, j]


In [29]:
#SVD to reduce features created by LSA and vectorizer
svd = TruncatedSVD(1000)
lsa = make_pipeline(svd, Normalizer(copy=False))
# Run SVD on the training data, then project the training data.
x_train_lsa = lsa.fit_transform(x_train_tfidf)

variance_explained=svd.explained_variance_ratio_
total_variance = variance_explained.sum()
print("Percent variance captured by all components:",total_variance*100)

#Looking at what sorts of paragraphs our solution considers similar, for the first five identified topics
paras_by_component=pd.DataFrame(x_train_lsa,index=vector_x_train)

for i in range(5):
    print('Component {}:'.format(i))
    print(paras_by_component.loc[:,i].sort_values(ascending=False)[0:10])

Percent variance captured by all components: 78.1693821799
Component 0:
text
I recently had the privilege of working closely with two men of uncommon courage  Andrew F. Brimmer, former chair of the D.C. financial control board, and Julius W. Becton Jr., former chief executive officer and superintendent of the D.C. public schools.Before taking up their work in the District, these men had climbed the heights of their chosen professions. Brimmer was one of the leading economists in the United States and a member of the Federal Reserve Board. Becton had retired, after 40 years of service, as a U.S. Army lieutenant general. Both Brimmer and Becton are members of what Thomas Jefferson called that "natural aristocracy," based not upon birth but upon "virtue and talents."Brimmer and Becton volunteered for arduous and time-consuming positions in our community in a time of troubles. They did this not for recognition, of which each already had received a full measure, but because they hoped to ma

Name: 0, dtype: float64
Component 1:
text
It was a bumpy year for So Others Might Eat, the District-based charity known as SOME, which provides food, clothing, medical care and other services to those in need.Just months before the charity was to open a glossy building in Northwest Washington, the Sept. 11 attacks occurred, and SOME's donations plummeted 30 percent as Americans turned their attention - - and wallets  to that cause. Then came the anthrax postal scare, and the charity group's mail dried up for several weeks.For a nonprofit organization that gets more than half its funding in small donations through the mail, it was a staggering one-two punch, said development coordinator Shonna Milliken Humphrey.But last week, the 30-year-old organization finally got some good news. Employees participating in the workplace fundraising campaign of the United Way of the National Capital Area had designated $1.4 million for SOME  a whopping 40 percent more than in the previous year. The new

Name: 1, dtype: float64
Component 2:
text
Robert L. Ehrlich Jr. said Tuesday that he will try to win his old job back in November, confirming plans for a much-anticipated rematch with Maryland Gov. Martin O'Malley, who defeated him in a bitter contest four years ago.A formal announcement, planned for next Wednesday in Montgomery County, will set up a rematch between two of the state's dominant political personalities.Both are fierce competitors. Their 2006 race was highly negative and at times intensely personal, with Ehrlich (R) repeatedly calling O'Malley "a whiner" and running ads that highlighted the homicide rate and struggling schools in Baltimore, where O'Malley was mayor. O'Malley (D) accused Ehrlich of dirty tricks and portrayed him as cozy with energy lobbyists and an unpopular President George W. Bush.Ehrlich said Tuesday that he plans to run a forward-looking campaign, disputing characterizations of the race as a grudge match."The last thing that people want to see is a sch

Name: 2, dtype: float64
Component 3:
text
FedEx Corp. threw some cold water on hopes that holiday shoppers would give the uncertain economic outlook a boost.The Memphis, Tenn., shipping giant  regarded as an economic gauge because it handles an average of more than six million packages a day across the world  reduced its profit target for the year because of an economic outlook pinched by the weak U.S. housing market and higher energy prices. The company said that as a result, it would ratchet back on capital spending."We believe U.S. [gross domestic product] will grow less than 3%" for the rest of FedEx's fiscal year, which ends May 30, said FedEx Chief Financial Officer Alan B. Graf Jr. He said the company lowered its profit guidance "almost entirely based on freight market" and economic forecasts.The reduced outlook overshadowed FedEx's results for the first quarter ended Aug. 31, which rose 4% from a year earlier. At 4 p.m. in New York Stock Exchange composite trading, shares of Fe

Name: 3, dtype: float64
Component 4:
text
FOR A TIME, there was a reassuring story to tell about the stock market and the economy. Stocks rose in a speculative frenzy, and then fell. But the U.S. economy recovered surprisingly quickly from Sept. 11, aided by the irrepressible American impulse to shop, plus a round of interest-rate cutting. The market was sick. The economy wasn't. Silicon Valley paper millionaires stopped buying Porsches. The rest of us kept buying Chevrolets.The magnitude, the speed and the circumstances of the stock-market decline are threatening that comforting story. Unless the market touches bottom soon  and despite yesterday's spectacular bounce in stock prices, there is no assurance of that  it is going to yank the economy down with it.We may not sink back into recession. But if we have a few more bad weeks on Wall Street, you can rip up those forecasts about the economy growing at a 3.5% annual pace in the second half. Instead, the economy will slow so much that

Name: 4, dtype: float64


In [30]:
#Create x train dataframe with features for modeling
vector_x_train_transformed = pd.DataFrame(x_train_lsa)

#Transform x_test
x_test_lsa = lsa.transform(x_test_tfidf)

#Create dataframe with x test transformed features
vector_x_test_transformed = pd.DataFrame(x_test_lsa)

In [31]:
# Random Forest on vectorizer features

random_forest_model(vector_x_train_transformed,vector_y_train,vector_x_test_transformed,vector_y_test)

Training set score: 0.976666666667

Test set score: 0.81875
[ 0.80880975  0.80880975  0.81332083]
